    Ben Christensen
    Math 345
    10/24/17
    
  

Implement the QR Decomposition of a matrix with several methods.

In [1]:
import numpy as np
from scipy import linalg as la
import pdb

# Problem 1
def qr_gram_schmidt(A):
    """Compute the reduced QR decomposition of A via Modified Gram-Schmidt.

    Parameters:
        A ((m,n) ndarray): A matrix of rank n.

    Returns:
        Q ((m,n) ndarray): An orthonormal matrix.
        R ((n,n) ndarray): An upper triangular matrix.
    """
    m,n = np.shape(A)
    Q = np.copy(A)
    R = np.zeros((n,n))
    for i in range(n):
        R[i,i] = la.norm(Q[:,i])
        Q[:,i] = Q[:,i]/R[i,i]
        for j in range(i+1, n):
            R[i,j] = Q[:,j] @ Q[:,i]
            Q[:,j] = Q[:,j] - R[i,j]*Q[:,i]
    return Q,R


# Problem 2
def abs_det(A):
    """Use the QR decomposition to efficiently compute the absolute value of
    the determinant of A.

    Parameters:
        A ((n,n) ndarray): A square matrix.

    Returns:
        (float) the absolute value of the determinant of A.
    """
    return np.prod(np.diag(qr_gram_schmidt(A)[1]))


# Problem 3
def solve(A, b):
    """Use the QR decomposition to efficiently solve the system Ax = b.

    Parameters:
        A ((n,n) ndarray): An invertible matrix.
        b ((n, ) ndarray): A vector of length n.

    Returns:
        x ((n, ) ndarray): The solution to the system Ax = b.
    """
    Q, R = qr_gram_schmidt(A)
    y = Q.T @ b
    n = np.shape(b)[0]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = y[i]
        for j in range(n-1, i, -1):
            x[i] = x[i] - R[i, j]*x[j]
        x[i] = x[i] / R[i, i]
    return x


# Problem 4
def qr_householder(A):
    """Compute the full QR decomposition of A via Householder reflections.

    Parameters:
        A ((m,n) ndarray): A matrix of rank n.

    Returns:
        Q ((m,m) ndarray): An orthonormal matrix.
        R ((m,n) ndarray): An upper triangular matrix.
    """
    m,n = np.shape(A)
    R = np.copy(A)
    Q = np.eye(m)
    for k in range(n):
        u = np.copy(R[k:, k])
        s = np.sign(u[0])
        sign = lambda x: 1 if x >= 0 else -1
        u[0] = u[0] + sign(s)*la.norm(u)
        u = u/la.norm(u)
        R[k:, k:] = R[k:, k:] - (2 * (np.outer(u, np.dot(u, R[k:, k:]))))
        Q[k:, :] = Q[k:, :] - (2 * (np.outer(u, np.dot(u, Q[k:, :]))))
    return Q.T, R


# Problem 5
def hessenberg(A):
    """Compute the Hessenberg form H of A, along with the orthonormal matrix Q
    such that A = QHQ^T.

    Parameters:
        A ((n,n) ndarray): An invertible matrix.

    Returns:
        H ((n,n) ndarray): The upper Hessenberg form of A.
        Q ((n,n) ndarray): An orthonormal matrix.
    """
    m,n = np.shape(A)
    H = np.copy(A)
    Q = np.eye(m)
    for k in range(n-2):
        u = np.copy(H[k+1:, k])
        s = np.sign(u[0])
        sign = lambda x: 1 if x>= 0 else -1
        u[0] = u[0] + sign(s) * la.norm(u)
        u = u / la.norm(u)
        H[k+1:, k:] = H[k+1:, k:] - 2 * np.outer(u, np.dot(u, H[k+1:, k:]))
        H[:, k+1:] = H[:, k+1:] - 2*np.outer(np.dot(H[:, k+1:], u), u)
        Q[k+1:, :] = Q[k+1:, :] - 2*np.outer(u, np.dot(u, Q[k+1:, :]))
    return H, Q.T

In [2]:
A = np.random.random((8, 8))
H, Q = hessenberg(A)
print(np.allclose(np.triu(H, -1), H))
print(np.allclose(Q @ H @ Q.T, A))

True
True
